# Reinforcement Learning with Keras interface: DQN

The goal of this notebook is to demonstrate how easy it is to do reinforcement learning with OpenMined and PySyft using the Keras interface. We will apply DQN to the game MountainCar-V0 from gym. The original code was written by Yash Patel. The original code can be found [here](https://towardsdatascience.com/reinforcement-learning-w-keras-openai-dqns-1eed3a5338c).

In [14]:
import syft
from syft import FloatTensor

In [15]:
import gym
import numpy as np
import random

from syft.interfaces.keras.models import Sequential
from syft.interfaces.keras.layers import Dense, Dropout
from syft.interfaces.keras.optimizers import SGD

from collections import deque
from syft import FloatTensor

In [16]:
class DQN:
    def __init__(self, env):
        self.env     = env
        self.memory  = deque(maxlen=2000)
        
        self.gamma = 0.85
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.005
        self.tau = .125

        self.model        = self.create_model()
        self.target_model = self.create_model()

    def create_model(self):
        model   = Sequential()
        state_shape  = self.env.observation_space.shape
        model.add(Dense(24, input_shape=state_shape[0], activation="relu"))
        model.add(Dense(48, activation="relu"))
        model.add(Dense(24, activation="relu"))
        model.add(Dense(self.env.action_space.n))
        model.compile(loss='mean_squared_error',
                      optimizer=SGD(lr=0.01), metrics =[])
        return model

    def act(self, state):
        self.epsilon *= self.epsilon_decay
        self.epsilon = max(self.epsilon_min, self.epsilon)
        if np.random.random() < self.epsilon:
            return self.env.action_space.sample()
        return np.argmax(self.model.predict(state).to_numpy())

    def remember(self, state, action, reward, new_state, done):
        self.memory.append([state, action, reward, new_state, done])

    def replay(self):
        batch_size = 5
        if len(self.memory) < batch_size: 
            return
        samples = random.sample(self.memory, batch_size)
        for sample in samples:
            state, action, reward, new_state, done = sample
            target = self.target_model.predict(state).to_numpy()
            if done:
                target.data[0][action] = reward
            else:
                Q_future = self.target_model.predict(new_state).max()
                target[0][action] = (Q_future * self.gamma + reward).to_numpy()[0]
                Q_future.delete_tensor = True
            self.model.fit(state, target, batch_size=1,epochs=1,verbose=True,validation_data=None)


    def target_train(self):
        weights = self.model.get_weights()
        target_weights = self.target_model.get_weights()
        for i in range(len(target_weights)):
            target_weights[i] = weights[i] * self.tau + target_weights[i] * (1 - self.tau)

    #def save_model(self, fn):
    #    self.model.save(fn)

def main():
    env     = gym.make("MountainCar-v0")
    gamma   = 0.9
    epsilon = .95

    trials  = 3
    trial_len = 4

    # updateTargetNetwork = 1000
    dqn_agent = DQN(env=env)
    steps = []
    for trial in range(trials):
        cur_state = FloatTensor(env.reset().reshape(1,2),autograd=True)
        for step in range(trial_len):
            action = dqn_agent.act(cur_state)
            new_state, reward, done, _ = env.step(action)

            # reward = reward if not done else -20
            # MOD - convert new_state to FloatTensor
            new_state = FloatTensor(new_state.reshape(1,2),autograd = True)
            dqn_agent.remember(cur_state, action, reward, new_state, done)
            
            dqn_agent.replay()       # internally iterates default (prediction) model
            dqn_agent.target_train() # iterates target model

            cur_state = new_state
            if done:
                break
        if step >= 2:
            print("Failed to complete in trial {}".format(trial))
            #if step % 10 == 0:
            #    dqn_agent.save_model("trial-{}.model".format(trial))
        else:
            print("Completed in {} trials".format(trial))
            #dqn_agent.save_model("success.model")
            break

if __name__ == "__main__":
    main()


Exception ignored in: <object repr() failed>
Traceback (most recent call last):
  File "/Users/yanndupis/anaconda2/envs/tflearn/lib/python3.5/site-packages/syft-0.1.0-py3.5.egg/syft/tensor.py", line 532, in __del__
  File "/Users/yanndupis/anaconda2/envs/tflearn/lib/python3.5/site-packages/syft-0.1.0-py3.5.egg/syft/tensor.py", line 1504, in delete_tensor
  File "/Users/yanndupis/anaconda2/envs/tflearn/lib/python3.5/site-packages/syft-0.1.0-py3.5.egg/syft/tensor.py", line 1492, in no_params_func
  File "/Users/yanndupis/anaconda2/envs/tflearn/lib/python3.5/site-packages/syft-0.1.0-py3.5.egg/syft/tensor.py", line 1474, in params_func
  File "/Users/yanndupis/anaconda2/envs/tflearn/lib/python3.5/site-packages/syft-0.1.0-py3.5.egg/syft/controller.py", line 143, in send_json
Exception: Unity Error: System.Collections.Generic.KeyNotFoundException: The given key was not present in the dictionary.
  at System.ThrowHelper.ThrowKeyNotFoundException () [0x00000] in /Users/builduser/buildslave/mon

Exception: Unity Error: System.Collections.Generic.KeyNotFoundException: The given key was not present in the dictionary.
  at System.ThrowHelper.ThrowKeyNotFoundException () [0x00000] in /Users/builduser/buildslave/mono/build/mcs/class/referencesource/mscorlib/system/throwhelper.cs:68 
  at System.Collections.Generic.Dictionary`2[TKey,TValue].get_Item (TKey key) [0x0001e] in /Users/builduser/buildslave/mono/build/mcs/class/referencesource/mscorlib/system/collections/generic/dictionary.cs:183 
  at OpenMined.Syft.Tensor.Factories.FloatTensorFactory.Get (System.Int32 id) [0x00008] in /Users/yanndupis/Documents/Data_Science/Encrypted_DL/OpenMined/UnityProject/Assets/OpenMined/Syft/Tensor/Factories/FloatTensorFactory.cs:24 
  at OpenMined.Syft.Tensor.FloatTensor.HookGraph (OpenMined.Syft.Tensor.FloatTensor& result, System.String creation_op, System.Boolean inline, System.Single scalar_input, OpenMined.Syft.Tensor.FloatTensor[] tensor_inputs, System.Int32[] resultShape, System.Single[] res

In [17]:
a = FloatTensor(np.array([3,5,2]))

In [18]:
a.max()
a.max()
#a.max()

Exception: Unity Error: System.Collections.Generic.KeyNotFoundException: The given key was not present in the dictionary.
  at System.ThrowHelper.ThrowKeyNotFoundException () [0x00000] in /Users/builduser/buildslave/mono/build/mcs/class/referencesource/mscorlib/system/throwhelper.cs:68 
  at System.Collections.Generic.Dictionary`2[TKey,TValue].get_Item (TKey key) [0x0001e] in /Users/builduser/buildslave/mono/build/mcs/class/referencesource/mscorlib/system/collections/generic/dictionary.cs:183 
  at OpenMined.Syft.Tensor.Factories.FloatTensorFactory.Get (System.Int32 id) [0x00008] in /Users/yanndupis/Documents/Data_Science/Encrypted_DL/OpenMined/UnityProject/Assets/OpenMined/Syft/Tensor/Factories/FloatTensorFactory.cs:24 
  at OpenMined.Syft.Tensor.FloatTensor.HookGraph (OpenMined.Syft.Tensor.FloatTensor& result, System.String creation_op, System.Boolean inline, System.Single scalar_input, OpenMined.Syft.Tensor.FloatTensor[] tensor_inputs, System.Int32[] resultShape, System.Single[] res